# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [176]:
import pickle

In [270]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [193]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [194]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [271]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[2306:3041]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=99.0.4844.74)
Stacktrace:
Backtrace:
	Ordinal0 [0x0112CF43+2608963]
	Ordinal0 [0x010BE8C1+2156737]
	Ordinal0 [0x00FB4190+1065360]
	Ordinal0 [0x00FA66A3+1009315]
	Ordinal0 [0x00FA638F+1008527]
	Ordinal0 [0x00FA59A0+1005984]
	Ordinal0 [0x00FA48D0+1001680]
	Ordinal0 [0x00FA4ED8+1003224]
	Ordinal0 [0x00FA4E68+1003112]
	Ordinal0 [0x00FAAB55+1026901]
	Ordinal0 [0x00FAAD96+1027478]
	Ordinal0 [0x00FA608D+1007757]
	Ordinal0 [0x00FA65A5+1009061]
	Ordinal0 [0x00FA638F+1008527]
	Ordinal0 [0x00FA59A0+1005984]
	Ordinal0 [0x00FA48D0+1001680]
	Ordinal0 [0x00FA4ED8+1003224]
	Ordinal0 [0x00FA4E68+1003112]
	Ordinal0 [0x00FAAFF0+1028080]
	Ordinal0 [0x00FA608D+1007757]
	Ordinal0 [0x00FA65A5+1009061]
	Ordinal0 [0x00FA638F+1008527]
	Ordinal0 [0x00FA59A0+1005984]
	Ordinal0 [0x00FA48D0+1001680]
	Ordinal0 [0x00FA4ED8+1003224]
	Ordinal0 [0x

In [333]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9791162850442,[10.0],"[100%, 0%, 0%, 0%, 0%]","[나는 죽는 것보다 살찌는게 더무서웠다, 나는 죽는 것보다 살찌는 게 더 무서웠다,...","[라**스, r****8, p********3, b******8, y********...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n☆부제:저자의 용기의 책.♧지금은 먹방이라는 주제로 유튜브나 TV에서 많은 방...
1,9788932321646,[9.6],"[75%, 25%, 0%, 0%, 0%]","[자연에 위로받고, 자연에 감사하게 되는 책, 식물을 따라 걷는 마음, 나는 식물을...","[s****4, b******e, s*****7, 유*]","[5점, 5점, 5점, 4점]",[\n'나는 식물을 따라 걷기로 했다' 제목이 이 책을 잘 표현해주고 있다고 생각한...
2,9791185889573,[8.0],"[67%, 0%, 0%, 33%, 0%]","[지금도 변하지 않는 성인의 지혜, 탈무드, [탈무드] 유대인 5000년 지혜의 원...","[로**랑, s*****4, j******3]","[5점, 5점, 2점]","[\n어렸을 때 ‘어린이 탈무드’로 탈무드를 처음 접했는데, 당시에 스토리 위주로 ..."
3,9791160402759,[9.6],"[83%, 11%, 6%, 0%, 0%]",[온 마음을 쏟을 수 있을 그 무언가를 갖고 있다면 [산문-온 마음을 다해 디저트]...,"[책****벤, 돼**스, i******o, 블**벳, w****9, 얼****험,...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n음식 관련 에세이가 읽고 싶어 찾아낸 책이다. 작가 이름과 작가의 그림이 낯익...
4,9791155812013,[9.5],"[80%, 20%, 0%, 0%, 0%]","[[Review] 타샤의 계절 Tasha Tudor Book of Holidays,...","[유*님, p*****3, 사*****날, 화**나, p*****3, 글***재, ...","[5점, 5점, 4점, 4점, 5점, 5점, 5점, 5점, 5점, 5점]","[\n\n 타샤의 계절\n \n올 연말연시는 조용한 한파처럼 지나갔는데, 이..."
...,...,...,...,...,...,...,...
100,9791159920998,[10.0],"[100%, 0%, 0%, 0%, 0%]","[에이즈와 맞서 싸운 사진 작가, 에르베 기베르]",[사**기],[5점],[\n 프랑스의 에르베 기베르는 에이즈에 관한 \n많은 글을 남겼다. \n그는 \n...
101,9791188345410,[9.3],"[86%, 7%, 7%, 0%, 0%]","[실은 괜찮지 않았던 날들 - 가린 에세이, 따뜻한 책, 내 마음속 외로운 아이에게...","[얼****험, d*******1, 북*, s******3, 배*정, b*****g...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n 말없이 상처를 삼키는 시간들그 순간을 견뎌낸 나를 위한 공감의 메시지 ...
102,9791188605033,[7.8],"[20%, 60%, 20%, 0%, 0%]","[잡지를 보지 않는다면, 결코 만날 수 없다 - [아무튼, 잡지]를 읽고, [아무튼...","[흙******에, 키*, y****i, k******i, C***e]","[4점, 5점, 4점, 3점, 3점]","[\n잡지를 보지 않는다면, 결코 만날 수 없다\n<아무튼, 잡지>를 읽고\n \n..."
103,9788954636599,[8.9],"[65%, 24%, 12%, 0%, 0%]","[[헤밍웨이의 작가 수업] 키웨스트와 아바나에서의 일 년, [서평]헤밍웨이의 작가수...","[북*더, 떨**o, y****7, 다*, b*****6, 봉*스, j*****n,...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n『노인과 바다(1952)』를 통해서 퓰리처상과 노벨문학상을 수상한 미국의 소설...


In [334]:
for i in range(2405, 2420):
    if isbn13[i] == '9788974431006':
        print(i)

2413


In [276]:
user.to_pickle('user_2306-2413.pkl', protocol = 4)

In [243]:
isbn13[347]

'9791191560091'

In [337]:
user1 = pd.read_pickle('user_1171-2413.pkl')
user2 = pd.read_pickle('user_2414-2596.pkl')
#user3 = pd.read_pickle('user_1700-2413.pkl')
#user4 = pd.read_pickle('user_2306-2413.pkl')
#user5 = pd.read_pickle('user_423-585.pkl')

In [338]:
df = pd.concat([user1, user2],ignore_index=True)
df.to_pickle('user_1171-2596.pkl', protocol = 4)

In [306]:
isbn13[2413] == '9788974431006'

True

In [303]:
for i in range(0,2024):
    if isbn13[i] == '9788996831266':
        print(i)

1673


In [330]:
df

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9791190710343,[9.4],"[67%, 33%, 0%, 0%, 0%]","[자기만의 방 - 버지니아 울프의 여성에세이, #여성에세이#자기만의 방, 자기만의 방]","[s*****0, 녹**다, m*******n]","[5점, 5점, 4점]",[\n\n \n시대를 뛰어넘어 진정한 페미니즘 비평의 장을 연 고전 <자기만의 방>...
1,9791190710350,[10.0],"[100%, 0%, 0%, 0%, 0%]",[오만과 편견],[할**장],[5점],[\n\n \n오만과 편견은 1813년 출판된 이래 한국에도 여러 출판사를 통해 번...
2,9791190710367,[10.0],"[100%, 0%, 0%, 0%, 0%]","[프랑켄슈타인, 엄청 기대되네요]","[책*레, m******4]","[5점, 5점]","[\n\n신이여,\n내가 그대에게 진흙으로 빚어달라 청했습니까?\n나를 어둠에서 끌..."
3,9791128858338,[],[],[],[],[],[]
4,9788954685603,[],[],[],[],[],[]
...,...,...,...,...,...,...,...
924,9791189385262,[10.0],"[100%, 0%, 0%, 0%, 0%]",[좋은 책입니다],[s********7],[5점],[\n쉽게 읽히고 유학 생활에서 오는 어려움을 작가가 위트있게 때로는 현실적으로 생...
925,9791168362147,[],[],[],[],[],[]
926,9791162143896,[10.0],"[100%, 0%, 0%, 0%, 0%]","[사랑과 이별의 에세이, 차재이-새벽은 이별에게 가혹하고, 코로나 시기에 이별로 가...","[초**떼, 버*, 소*, c***6, -*, -*, m***l, b******g,...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n\n\n \n차재이님은 배우로도 활약하고 있는 분이다. <새벽은 이별에게 가혹...
927,9791166496196,[],[],[],[],[],[]


In [329]:
isbn13[0] == '9791190710343'

True